# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
     # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, 1000000);")

    # Wait to load the page.
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        print('到達頁面底端...')
        break

    last_height = new_height
    print('網頁更新中...')
    
# From https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端...


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 豪宅回春？都更分回案「明日博」降價2次後　首度標脫

▲國產署6次列標、兩度降價的「明日博」終於賣出去了。（圖／記者宋良義攝）
記者吳靜君／台北報導
財政部國產署開標都更分回宅，這次列標32戶、標脫6戶，其中屢標屢敗的「碧湖君鄰」、「明日博」在再度降價求售之後，這次終於各標脫兩戶，明日博總價上億元、單價也破百萬元，這次首度標脫引發業界的關注、碧湖君鄰賣出最後兩戶店面之後，也正式完售。
根據國產署的公告，這次公告列標共計32戶，其中三重區的「翔譽雙子星」、大安區的「忠泰鳳磐」、大同區的「永德言葉」各列標1戶；萬華區的「全坤威峰」、中正區的「正隆天地」各列標2戶；板橋「昇陽文薈」列標6戶以及大安區的「明日博」列標17戶，共計32戶。
這次，標脫的建案有大安區的忠泰鳳磐，面積為54坪、標脫金額為4623萬元，換算每坪價格85萬元；碧湖君鄰兩戶皆為店面，面積大約37、37坪左右，標脫金額在3006萬元，每坪價格在81萬元。
另外，明日博也都是大坪數一戶是97.1坪與120坪，標脫金額分別是9646萬與1億3515萬元，每坪單價上看100萬到112萬元；永德言葉則是49坪，得標金額3443萬元，每坪約70萬元。
根據財政部國產署顯示，「最難賣」的兩個建案終於在歷次降價之後，終於賣出去了，其一就是「碧湖君鄰」的兩個店面，列標11次，經歷過4次降價終於標脫，溢價率約0.04%到0.17%之間；明日博也列標6次，降價兩次之後終於標脫兩戶，且單價仍創百萬身價，溢價率則是在0.01%到0.02%。
而永德言葉是這一批都更分回案中，溢價率最高者，溢價率達到5.17%。財政部國產署表示，現在已經「完售」的都更分回案除了碧湖君鄰外、永德言葉、忠泰鳳磐也已經完售。財政部表示，下一批都更分回案將在7月底再度公告，如果有興趣的民眾仍可持續關注。
----------------------------------------------------------------------

[1] 理財周刊／非農好數據　美元不樂觀

文／理周教育學苑．方鈞
Q：美國六月非農就業人口關鍵數據公布大大優於預期，對於市場而言，是極大的鼓舞士氣，顯示經濟活動已

▲盛弘集團董事長楊弘仁。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
盛弘醫藥（8403）今（10）日公告6月合併營收3.5億元，年增34.31%。盛弘指出，台灣疫情趨緩並解封，政府推動防疫新生活，防疫物資需求持續，本業醫藥流通事業部、躍獅連鎖藥局及旗下熔噴布大廠敏成(4431)持續挹注；累計今年上半年合併營收17.9億元，年增16.59%。
盛弘表示，儘管全球疫情並未停歇，但台灣防疫有成且全面解封，政府倡導防疫新生活，民眾在外出時更重視清潔衛生，保健及防疫產品需求不減，帶動盛弘醫藥流通事業部及躍獅連鎖藥局營運穩步發展。
盛弘指出，解封後民眾生活恢復正常，醫療及健診人數逐漸回流，健康管理部門如健檢及健康管理之營運陸續恢復，旗下精準健康公司及哈佛健診除高階健檢外，也推出多項基因檢測服務，目標預防醫學市場，將預防治療的時間提前至未發病階段。
盛弘說明，轉投資的敏成（4431）為台灣熔噴不織布大廠，自年初以來持續生產口罩用熔噴布供國內防疫所需，為使重要原料熔噴不織布供應無虞，敏成已在月初啟用新產線，產能除持續配合政府定額徵用供國內防疫使用到年底，其餘產量將售予國內廠商，以及消化國外客戶的訂單。
盛弘強調，國發會5月發表「後COVID-19台灣經濟發展對策」報告，提出七支箭重點對策，第一支箭即是打造大健康產業，以創新科技結合健康醫療領域應用的數位療法、智慧醫療、精準醫療、數位防疫等，驅動台灣大健康產業的推進。
►過期票券別浪費！簡單3步驟　教你換1.5倍東森幣
----------------------------------------------------------------------

[9] 金融海嘯以來最糟5月！受雇員工比去年減少千分之4　月減1萬7千人

▲主計總處國勢普查處副處長陳惠欣。（圖／記者余弦妙攝）
記者余弦妙／台北報導
對此，主計總處國勢普查處副處長陳惠欣表示，隨著疫情逐漸趨緩，受雇員工人數雖然持續減少，不過從細部產業別觀察5月份在餐飲、住宿、休閒減幅緩，不過部分製造業月減人數與上期相同8千人，主要是受到國際疫情影響所致，接下來會持續觀察。
►過期票券別浪費！點這邊...幫你搶救變身
----------------------------------------------------------------------

[10] 日本機

▲2020年第二季蘋果Mac的出貨量將增長5.1%。。（圖／達志影像／美聯社）
記者林鼎閎／綜合報導
根據Gartner最新的PC出貨量報告顯示，2020年第二季全球PC出貨量將增長2.8%，蘋果Mac的出貨量將增長5.1%。數據顯示，今年4至6月底（第二季），蘋果估計將售出逾436萬台Mac，高於去年同期的415萬台。
在經過新冠肺炎破壞了供應鏈並抑制了全球需求之後，目前個人電腦市場開始呈現出復甦的跡象。 儘管該病毒在美國仍在流行，但大多數發達國家已經設法讓疫情趨緩。
根據釁蘋果在2020年第二季度的Mac出貨量預估為436.8萬台，高於2019年第二季度的415.7萬台，即使在全球疫情的背景下，其出貨量也在保持增長。
本季度蘋果的市佔率從去年同期的6.6%增長到6.7%。在蘋果出貨量上升的同時，整個PC市場的增長也達到了2.8%。
----------------------------------------------------------------------

[22] 全國數位改選董監事　練台生人馬估佔2董1監席次

----------------------------------------------------------------------

[23] 預計增50億產值！「3步驟」成為藝FUN券店家　最快3天審查通過

▲ 「藝FUN券」將於7/22開放使用，文化部正招募藝文業者加入。（圖／文化部）
記者林育綾／台北報導
除了行政院發行振興三倍券，文化部加碼12億推出每份600元的「藝FUN券」，總計發行200萬份，今（10）日起招募藝文業者加入，只要「3步驟」就能成為「藝FUN店家」，最快3個工作天即可收到審查結果。
▲文化部招募藝文業者加入，只要「3步驟」就能成為「藝FUN店家」。（圖／文化部提供）
「藝FUN券」APP在民眾使用時，將能即時露出附近可使用「藝FUN券」的藝文場域及店家，提供民眾消費選擇。消費時只要讓民眾掃描店家QR Code或店家掃描民眾付款QR Code，即完成付款；此外，為能即時紓解店家金流壓力，特別設計「天天對帳、週週付款」機制，簡化店家申領款項相關程序。
▲文化部鼓勵藝文業者推出「三倍券」、「藝FUN券」雙券合用方案，擴大藝文消費商機。（圖／文化部提供）
藝FUN券以「藝文專用、不限區域使用」為原則，藝

▲東森集團總裁王令麟率主管團隊參訪醒吾科大，洽談結盟讓學生用專業打造第一桶金。（圖／東森購物提供，以下同）
記者洪菱鞠／台北報導
東森集團總裁王令麟於今年4月率團隊參訪醒吾科大後，雙方隨即展開各項深度合作事宜，並由新型態的社群電商開始，希望讓學生運用科技及專業，在校期間就能從事雲端創業。王令麟表示，學貸壓力與打工賺錢成為許多大學生面臨的困境，但壓力是成長的動力，期望能善用社群電商平台「東森購享賺」，勇敢逐夢，在未來能產生許多年輕的創業家。
東森購物6月於醒吾科大舉辦「社群電商：東森購享賺」說明會，逾百位有興趣的學生報名參加；7月再次舉辦「東森購享賺：社群經營與電商操作」說明會，進一步提出東森集團未來與校園合作之新型態社群電商新商模，吸引近百位教授及新媒體行銷領域專家參與，現場討論踴躍，場面相當熱烈。
▲「東森購享賺：社群經營與電商操作」說明會，吸引醒吾科大近百位教授及新媒體行銷領域專家參與。
▲對於與東森集團的合作，醒吾科大校長陳義文表示期待。
醒吾科大校長陳義文認為，電子商務是醒吾目前發展的重點項目之一，除了學校相關配套課程，實作機會也很重要，此次與東森集團合作社群電商「東森購享賺」，讓每個參與的學生自己就能雲端創業。透過暑期大型教師聚會，醒吾也將與相關教授研習討論，培訓學生全方位的社群經營能力，整合學術專業與商業應用，推出有益師生的社群電商跨界課程與活動。
陳義文表示，感謝東森集團選擇醒吾作為合作夥伴，提供給學生這麼好的機會，期許未來雙方能在其他項目展開更密切的合作，進一步深化產學間的鏈結，也幫助所有醒吾的學生在未來取得更具優勢的職場競爭力。
▲東森集團於醒吾科大展開各項深度合作，日前邀請東森購物營運長顧武男針對「東森購享賺」召開說明會，提出東森集團未來與校園合作之新型態社群電商新商模。
東森購物營運長顧武男說，近幾年隨著數位行動網路用戶、行為的增加，以及數位影音跨界人才的趨勢，現今年輕學子想要打工賺錢擺脫學貸沈重的壓力，透過東森與醒吾的合作，善用社群電商「東森購享賺」，等於用集團的力量幫助排除多數難題，學生只需要專注在經營社群「粉絲經濟」，透過分享商品及推薦機制，讓更多人來購買東森購物平台上的產品，就可以獲得相對應的現金回饋。
顧武男提到，「東森購享賺」零門檻不僅適合學生族群、社群電商著重社交互動的商業型態更是學生可以應用自如的地方，就算沒有受過專業訓練也能夠

▲新台幣強勁升值，超越日圓及韓元升值幅度，成為亞洲最強貨幣。（圖／記者李毓康攝）
●李沃牆／淡江大學財務金融學系專任教授及兩岸金融中心副主任，富華創投及兆豐第一創投董事、品豐投顧榮譽顧問
因應疫情所需，主要國家央行大舉印鈔救市，撒錢紓困逐漸發酵；熱錢到處流竄，推升資產價格，早把疫情拋諸腦後。全球不少國家股市在資金狂潮下步步走升，如近日的美國那斯達克（Nasdaq）及費城半導體指數履創歷史新高、中國大陸的上證指數創下新2年來新高，深圳綜合指數則創下5年新高；具高股息的台股更獲外資青睞，也創下30年來新高，離史上最高的12,682已經愈來愈近。
外資累積淨賣超　為何台幣升不停
坦然言，今年受疫情影響，3月全球股市崩跌，外資於3月賣股匯出金額高達3,150億，創歷年單月最大匯出金額，也刷新台股史上最大單月賣超紀錄；結果3月台股指數下跌1,584點，跌幅逾14%；累積前5月，外資共賣超台股逾6,000億，但6月轉為買超800多億。
一.外資賣股後並未全數匯出，而是停泊在台，等待機會入市。
二.美元波段貶值：因聯準會（Fed）降息及實施無限QE，加上美國政府為抗疫而大幅增加財政開支；在財政及外貿雙赤字下，導致美元呈現弱勢。
三.再者，今年4月，美國前三大的海外債權國日本、中國、英國皆減持美債；其中，中國減持88億美元，這無非也是造成美元趨貶的因素之一。
四.放空美元避險：美元指數較近期高點下跌了6%，而期貨市場上的美元淨空倉規模接近2018年以來最高。
五.今年上半年美國大幅降息，加上國際金融市場動盪，金融業海外投資獲利了結匯回；外匯市場美元供給增加，也導致台幣升值。
六.另外，美中貿易戰中，台灣成避風港，外人對台直接投資增加，也使新台幣匯率相對獲得支撐。
▲市場普遍認為，29元整數關卡，應是「楊金龍防線」。（圖／記者黃克翔攝）
出口商苦不堪言　台幣有楊金龍防線？
美元會再貶，台幣還會升值？無疑是許多人心中的疑問。摩根士丹利亞洲前主席Stephen Roach撰文指出，美元作為世界主要儲備貨幣擁有「過度特權」的時代即將結束。美元可能會大幅下挫35%；但筆者認為此觀點似乎太過於主觀，可能性較低。
▲企業不能再過度依賴零成本的自然避險。（圖／記者李毓康攝）
匯率如丟骰子難測　靈活避險才是王道
外匯市場詭譎多變，匯率如丟骰子難測；央行即使進場干預阻升，也無法照顧所有的人。建議出口商

▲新台幣開盤貶值1.9分圖。（圖／記者湯興漢攝）
記者陳心怡／台北報導
台北外匯市場新台幣兌美元匯率今（10）日開盤貶值1.9分，以29.600元兌1美元開出。
新台幣匯率昨（9）日收在29.581元兌1美元，今天以29.600元兌1美元開出、貶值2.6分。
不過，在台積電領漲下，台股開盤漲57.75點，新台幣也跟著由貶轉升，早盤升1.21角，來到29.46元。
----------------------------------------------------------------------

[53] 台股開盤漲58點　指數站回12200點關卡　台積電衝破350元大關

▲台股以12250點開出。（資料照／記者湯興漢攝）
記者陳心怡／台北報導
美國各州新冠確診持續刷新高，使道瓊工業指數收盤大跌逾 300 點，回吐本周漲幅，不過，美科技股續強，使得美四大指數漲跌互見。台股今（10）日開盤上漲58點、以12250點開出。 
截至上午9點6分，指數來到12206點、上漲13點。
 蘋概三王中，台積電（2330）以352.5元開出、上漲2.17%；股王大立光（3008）以4075元開出、下跌7.49%；鴻海（2317）以86.2元平盤開出。
美股道瓊指數下跌 361.19 點，或 1.39%，收 25,706.09 點。標普 500 指數下跌 17.89 點，或 0.56%，收 3,152.05 點。那斯達克指數上漲 65.89 點，或 0.63%，收 10,558.39 點。費城半導體指數上漲 26.96 點，或 1.32%，收 2,076.72 點。
----------------------------------------------------------------------

[54] 大樂透加碼最終期25組100萬全送出！　1張50元挑戰歷年最高獎金

▲大樂透端午加碼百組百萬進入最後加開，2500萬獎金將一次送出。（示意圖／記者鄒鎮宇攝）
記者紀佳妘／台北報導
大樂透端午加碼100組100萬元，今（10）日將加開最後25組，2,500萬獎金將一次送出，而這次為歷年大樂透「100組100萬元」加碼，最後一期剩餘100萬獎項組數最多的一次，也就是說民眾抱走均分後的大樂透加碼100萬元獎項，有可能是歷年大樂透加碼「100組100萬元」最高的獎金

▲鴻海土城總部。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
鴻海（2317）今（9）日發出聲明，針對集團旗下子公司FII（工業富聯）員工不滿沒有0~6歲公司養一事，強調個別關聯子公司員工的薪資、福利、獎懲等等事宜，由子公司相關權責單位決定，鴻海公司無法代為決策。
鴻海創辦人郭台銘先前提出0~6歲國家養的政策，鴻海董事長劉揚偉跟著響應，親自在鴻海尾牙上宣布率先實施0~6歲公司養，但集團旗下子公司員工不滿沒有0~6歲公司養福利。
鴻海發出聲明表示，基於公司治理原則，個別關聯子公司員工的薪資、福利、獎懲等等事宜，由子公司相關權責單位決定，鴻海公司無法代為決策，但媒體所提有關子公司員工的意見，則會透過相關子公司的董事會溝通管道轉達。
據悉，當初鴻海對轉職員工的承諾是，現有的福利會比照鴻海維持不變，所以生育津貼、媽媽福袋都還是一樣，至於今年起實施的新政策，就不能追溯，而且FII（工業富聯）、FIT（鴻騰）、FIH（富智康）、亞太電信等關聯子公司，都有自己的營運考量，不可能全部跟進。
►過期票券別浪費！簡單3步驟　教你換1.5倍東森幣
----------------------------------------------------------------------

[67] 「防疫光電產品」帶動衝刺！光寶科6月營收142.87億元　創今年新高

▲光寶集團副董事長暨總執行長陳廣中。（圖／記者楊絡懸攝）
記者楊絡懸／台北報導
光寶科技（2301）6月份全球合併母子公司營收為142.87億元，月增4%，創今年月營收新高，但年減12%。受惠於核心事業終端市場需求穩定，累計今年上半年合併營收為743.37億元。
光寶科技指出，光電部門6月份營收佔整體的18%；其中，LED元件受惠於不可見光、5G與AIoT，與民生醫療、消毒與淨化之UV LED等應用需求提升，加上戶外LED照明產品出貨增加，帶動光電部門整體營收月增約近1成。
資訊產品部門營收佔比達67%；其中，雲端運算高階伺服器、網通設備電源管理系統以及筆記型電腦電源和鍵盤產品出貨持續成長，帶動資訊產品部門營收較上月持續成長。儲存裝置部門六月份佔整體營收達12%，營收月增約5%
光寶副董事長暨總執行長陳廣中表示，就經驗而言，第三季向來是傳統旺季，不過受新冠肺炎（COVID-19）疫情影響，「今年旺季不旺，是確定的事情。」第三

蘋果加速整合生態圈，加持台積電外，在全系列產品的硬體改變也不手軟，下半年蘋果新品齊發輪廓已相當明確下，蘋概供應鏈可望接棒大漲，玉晶光之後，最靚蘋概股即將出列！
文／林麗雪
經過美中貿易戰及新冠肺炎疫情的洗禮，全球消費性大廠蘋果的營運策略也在今年出現大轉彎，從整合生態圈、全系列產品改變設計、到加速轉移及分散全球生產基地，蘋果大動作的調整告一段落後，隨著下半年各項新品齊發的輪廓已愈來愈明確，相關供應鏈廠營運正蓄勢待發，蘋概股第三季重回投資人懷抱逐漸可期。
日前蘋果以預製錄影的方式發表ＷＷＤＣ開發者大會，針對旗下硬體發表一系列的系統升級，雖未發表新的Apple Watch或Mac產品，讓市場有些小失望，但執行長庫克證實蘋果將採用ARM架構，自主開發Mac晶片，這是蘋果加速整合生態圈，並使各產品之間的App更容易連繫的的重要策略。
拉貨行情一路旺到年底
蘋果掰了英特爾，和蘋果長期合作的台積電將是最大贏家，法人推測，蘋果將使用台積電的InFO封裝技術，有助於提振台積電的先進封裝產能，短期而言，蘋果轉換成ARM架構處理器的電腦市占率將不會太高，今、明年的貢獻不大，但中長期對台積電營運正向。
蘋果整合生態圈，也從硬體的改變著手。早在新冠肺炎疫情爆發之前，蘋果一系列的新品藍圖輪廓都已大致確立，包括新款智慧型手機iPhone 12系列確定將搭載屏下指紋辨識外，預估將於十一月發表的iPad Air 4平板，也將具有指紋辨識功能。
據了解，蘋果屏下指紋辨識功能將採用一至二顆的VCSEL感測器，該感測器的供應商確定為穩懋，此外，iPhone 12將是首款５Ｇ手機，先前穩懋法說會即已釋出，５Ｇ手機所需的ＰＡ數量，將比４Ｇ手機至少增加二～五顆以上，兩項應用加總，也無怪乎，穩懋對於第三季仍持營運暢旺的看法，上半年股價表現相對沉寂的穩懋，可以視為機構法人是否重新擁抱蘋概股的重要觀察指標。
不過，雖然一般預料蘋果有望在九月如期宣布新款智慧型手機上市，但iPhone鏡頭供應商大立光釋出，將自七月中旬開始出貨iPhone 12用的後置高階鏡頭，此時程較以往延遲了四到六周，雖然來得晚了些，但也代表整個供應鏈的拉貨行情，可以一路旺到年底。
其中，鏡頭二哥玉晶光在７Ｐ廣角鏡頭取得突破，預料將在iPhone 12取得更多後鏡頭訂單，今年到明年的成長性將超過大立光，六月份以來，玉晶光股價已大漲逾六成，蘋概題材早已

網搜小組／鄺郁庭報導
PTT名人、鄉民乾爹土豪哥（金庸）因為在八卦板上曝光了自己人生，從此一舉一動受到注目。有在投資股票的他，日前在自己創的閒聊股市社團中發文，「暫時先脫手了，近期為了土地買賣有點忙，持股僅剩特別股與少許金融，電信股全賣光了」，並分享他半年以來的對帳單，大賺2億讓網友都看呆。
▲金庸玩股票半年賺2億。（圖／翻攝自YouTube／gn金庸）
金庸在私密社團中發文，說他暫時先將股票都脫手，持股只剩特別股和少許金融股，「本金就是12億，反正有賺錢就是買桃園、新竹農地山坡地，當樂透投資。」並說自己選股進場的標準，大部分是「十年線就買，20週均線站上就買進，跌破就賣。」
另外，KD大於90也賣，布林特通道碰到上軌、量出歷史量也賣，底部無量就慢慢撿。「今年只要破3月前高的股票都會拉上一波，原相、鈺太、華擎甚至遊戲股」，金庸說近期他嘗試抱1、2天就賣短進短出，但發現自己好像不適合這樣操作，之後還是會找低基期的存股玩。
▲金庸之前收租收到450萬現鈔。（圖／翻攝自PTT）
他表示，目前買進力晶仍有減資換股的權力，提醒有力晶股票的人，到時候可以換力積電，「反正只要仔細看懂財報，有3做12吋廠、2座8吋廠，價值多少自己想吧！絕對一波興櫃蜜月。」
----------------------------------------------------------------------

[92] 生技業受到矚目　生技條例租稅獎勵有望加碼

▲生技新藥產業發展條例政府預計擴大租稅獎勵和延長時間，有利生技業發展。（圖／資料照）
記者吳靜君／台北報導
由於新冠肺炎（COVID-19）疫情影響，世界各國更加在意生技產業的發展，值此之際，台灣現行生技新藥產業發展條例規定將在明年落日，目前政府在廣納意見，盤點產業發展的需要，朝著擴大租稅獎勵適用範圍及延長條例適用期間邁進。
KPMG生技暨健康產業共同主持人蘇嘉瑞指出，預計修正的生技新藥條例的擴大適用，正符合生醫與電資整合的產業政策，例如2017年將新興生技醫藥產品納入細胞治療、基因治療和精準醫療，法規上也配合修正特管辦法，加入細胞治療技術及實驗室開發檢測（LDTs)等部分，而再生醫療製劑管理條例也將可望通過，台灣再生醫療與精準產業也將隨之發展。
蘇嘉瑞表示，這次預計開放的數位醫療（BIO-ICT)及國家策略生技產品，例如防疫國家隊，